In [108]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import re
import ast
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
pd.set_option('display.max_columns', 100)
pd.options.display.max_rows = 4000

In [109]:
#Letters to iterate through
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']
#Empty lists to store our data
names_list = []
links_list = []
start_list = []
end_list = []
pos_list = []

In [110]:
for letter in alphabet:
    #Link to webpage using requests and BeautifulSoup
    page = requests.get('https://www.basketball-reference.com/players/' + letter)
    soup = BS(page.content, 'html.parser')
    
    #Select The table then the rows
    table = soup.find('table', class_ = 'sortable stats_table')
    rows = table.findAll('tr')
    
    #Iterate through the rows and get all player names
    new_names_list = [row.find('th') for row in rows]
    new_names_list = [name.find('a') for name in new_names_list]
    new_names_list = [name for name in new_names_list if name]
    new_names_list = [name.string for name in new_names_list]
    
    #Iterate through the rows and get links to all player pages
    new_links_list = [row.find('th') for row in rows]
    new_links_list = [link.find('a') for link in new_links_list]
    new_links_list = [link for link in new_links_list if link]
    new_links_list = [link['href'] for link in new_links_list]
    
    #Iterate and get first year of a players career
    new_start_list = [row.findAll('td') for row in rows]
    new_start_list = [year for year in new_start_list if year]
    new_start_list = [year[0].string for year in new_start_list]
    
    #Iterate and get last year
    new_end_list = [row.findAll('td') for row in rows]
    new_end_list = [year for year in new_end_list if year]
    new_end_list = [year[1].string for year in new_end_list]
    
    #Iterate and get position
    new_pos_list = [row.findAll('td') for row in rows]
    new_pos_list = [pos for pos in new_pos_list if pos]
    new_pos_list = [pos[2].string.replace('-','') for pos in new_pos_list]
    
    #Add the new data to our lists
    names_list = names_list + new_names_list
    links_list = links_list + new_links_list
    start_list = start_list + new_start_list
    end_list = end_list + new_end_list
    pos_list = pos_list + new_pos_list
    
    
#Turn our lists into a dataframe and name the columns    
df = pd.DataFrame([names_list, links_list, start_list, end_list, pos_list]).transpose()
df.columns = ['Player', 'Link', 'StartYear', 'EndYear', 'Position']

In [111]:
#Change data types of years to int64 and creating a column for amount of time in league
df.StartYear = df.StartYear.astype('int64')
df.EndYear = df.EndYear.astype('int64')
df['Tenure'] = df.EndYear - df.StartYear

In [112]:
df_eligible = df[(df.StartYear >= 1985)]
df_eligible.reset_index(inplace = True, drop = True)
#Limiting data to players who entered the league during or after 1979 to include 3p% as a feature 


In [113]:
def get_player_info(href1):
    page = requests.get('https://www.basketball-reference.com'+str(href1))
    soup = BS(page.content, 'html.parser')
    table_body=soup.find_all('p')
    work=table_body[0:10]
    #print(work)
    height= re.findall('(\d*cm)', str(work))
    weight= re.findall('(\d*kg)', str(work))
    bday= re.findall('data-birth=\"(\d*\-\d*\-\d*)', str(work))
    #accolades
    accolades_body=soup.find(id="bling")
    All_Star_apps= re.findall('(\d*)x All Star', str(accolades_body))
    All_NBA_apps= re.findall('(\d*)x All-NBA', str(accolades_body))
    All_Def_apps= re.findall('(\d*)x All-Defensive', str(accolades_body))
    HOF= re.findall('(Hall of Fame)', str(accolades_body))
    empty_list=0
    if len(All_Star_apps)==empty_list:
        All_Star_apps='0'
    else:
        All_Star_apps=All_Star_apps[0]
    if len(All_NBA_apps)==empty_list:
        All_NBA_apps='0'
    else:
        All_NBA_apps=All_NBA_apps[0]
    if len(All_Def_apps)==empty_list:
        All_Def_apps='0'
    else:
        All_Def_apps= All_Def_apps[0]
    if len(HOF)==empty_list:
        HOF='0'
    else:
        HOF= HOF[0]
    bio= [height[0],weight[0],bday[0],All_Star_apps,All_NBA_apps,All_Def_apps,HOF]
    
    #Find Stats overview, then pull out individual stats then extract the stats and make them a list
    stats = soup.find('div', class_ = "stats_pullout")
    cells = stats.find_all('p')
    stats_list = list(map(lambda x : x.string, cells[3:23:2]))
    
    return bio + stats_list

In [10]:
info_list = [get_player_info(link) for link in df_eligible.Link]
#Very Slow

In [11]:
df_info = pd.DataFrame(info_list, columns=['Height','Weight', 'Birthday', 'All_Star_apps','All_NBA_apps','All_Def_apps','HOF', 'Games', 'PPG', 'TRPG', 'APG', 'FG_pct', '3_pt_pct', 'FT_pct', 'eFG_pct', 'PER', 'WS'])
df_eligible = pd.concat([df_eligible, df_info], axis = 1)
#Add new columns to our dataframe

In [12]:
df_eligible.head()

,Player,Link,StartYear,EndYear,Position,Tenure,Height,Weight,Birthday,All_Star_apps,All_NBA_apps,All_Def_apps,HOF,Games,PPG,TRPG,APG,FG_pct,3_pt_pct,FT_pct,eFG_pct,PER,WS
0,Alaa Abdelnaby,/players/a/abdelal01.html,1991,1995,FC,4,208cm,108kg,1968-06-24,0,0,0,0,256,5.7,3.3,0.3,50.2,0.0,70.1,50.2,13.0,4.8
1,Mahmoud Abdul-Rauf,/players/a/abdulma02.html,1991,2001,G,10,185cm,73kg,1969-03-09,0,0,0,0,586,14.6,1.9,3.5,44.2,35.4,90.5,47.2,15.4,25.2
2,Tariq Abdul-Wahad,/players/a/abdulta01.html,1998,2003,F,5,198cm,101kg,1974-11-03,0,0,0,0,236,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5
3,Shareef Abdur-Rahim,/players/a/abdursh01.html,1997,2008,F,11,206cm,102kg,1976-12-11,1,0,0,0,830,18.1,7.5,2.5,47.2,29.7,81.0,47.9,19.0,71.2
4,Álex Abrines,/players/a/abrinal01.html,2017,2019,GF,2,198cm,90kg,1993-08-01,0,0,0,0,174,5.3,1.4,0.5,38.7,36.8,88.0,52.5,8.8,5.0


In [13]:
df_eligible.to_csv('nba_ref_data.csv')
#export dataframe to a csv

In [148]:
def get_NCAA_href(href1):
    page = requests.get('https://www.basketball-reference.com'+str(href1))
    soup = BS(page.content, 'html.parser')
    link= re.findall('(https:\/\/www\.sports-reference\.com\/cbb\/players\/.*\.html)', str(soup))
    if len(link)>0:
        link= link[0]
        return link
    else:
        return

In [149]:
def get_hrefs(year):
    page = requests.get('https://www.basketball-reference.com/draft/NBA_'+str(year)+'.html')
    #print('here')
    soup = BS(page.content, 'html.parser')
    table_body=soup.find('tbody')
    rows = table_body.find_all('tr')
    hrefs=[]
    for row in rows:
        try:
            href=row.find_all('a')
            x= (href[2])
            x= re.findall('\"+(/players/+.*)\"', str(x))
            if len(x)>0:
                hrefs.append(x)
            else:
                pass
            continue
        except:
            pass
    return hrefs

In [151]:
hrefs2016= get_hrefs(2016)
website=[]
for href in hrefs2000:
    #time.sleep(.3)
    if website==0:
        a= get_NCAA_href(href[0])
        website.append(a)
    else:
        a= get_NCAA_href(href[0])
        website.append(a)
website

['https://www.sports-reference.com/cbb/players/ben-simmons-1.html',
 'https://www.sports-reference.com/cbb/players/brandon-ingram-1.html',
 'https://www.sports-reference.com/cbb/players/jaylen-brown-1.html',
 None,
 'https://www.sports-reference.com/cbb/players/kris-dunn-1.html',
 'https://www.sports-reference.com/cbb/players/buddy-hield-1.html',
 'https://www.sports-reference.com/cbb/players/jamal-murray-1.html',
 'https://www.sports-reference.com/cbb/players/marquese-chriss-1.html',
 'https://www.sports-reference.com/cbb/players/jakob-poeltl-1.html',
 None,
 'https://www.sports-reference.com/cbb/players/domantas-sabonis-1.html',
 'https://www.sports-reference.com/cbb/players/taurean-prince-1.html',
 None,
 'https://www.sports-reference.com/cbb/players/denzel-valentine-1.html',
 None,
 None,
 'https://www.sports-reference.com/cbb/players/wade-baldwin-1.html',
 'https://www.sports-reference.com/cbb/players/henry-ellenson-1.html',
 'https://www.sports-reference.com/cbb/players/malik-bea

In [152]:
def get_player_NCAA_stats(website):
    #Pull Final Year NCAA Stats
    if website== None:
        return 
    page = requests.get(website)
    soup = BS(page.content, 'html.parser')
    table_body=soup.find('tbody')
    if table_body == None:
        return 'error'
    rows = table_body.find_all('tr')
    data=[]
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        data.append(cols)
    column_names=['School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','Blank', 'SOS']
    column_names26=['School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','Blank', 'SOS']
    df = pd.DataFrame(data)
    if len(df.columns)==28:
        df.columns= column_names
    else:
        df.columns= column_names26
    df['Yrs_College']= len(df)
    df= df.tail(1)
    df.reset_index(inplace= True, drop= True)
    
    #Get Name
    table_body=soup.find_all(itemprop= "name")
    table_body
    name= re.findall('<h1 itemprop=\"name\">(.*)</h1>', str(table_body))
    if len(name)==0:
        name= re.findall('<span>(.*)</span>', str(table_body))
    else:
        pass
    df['Name']= name
    columns=['Name','Yrs_College','School', 'Conf', 'G', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS']
    df= df[columns]
    
    return df

In [153]:
def NCAA_All_Together(Class):
    hrefs= get_hrefs(Class)
    website=[]
    for href in hrefs:
        if website==0:
            a= get_NCAA_href(href[0])
            website.append(a)
        else:
            a= get_NCAA_href(href[0])
            website.append(a)
    switch= 0
    for player in website:
        player1= get_player_NCAA_stats(player)
        if isinstance(player1, str):
            continue
        elif switch == 0:
            df= player1
            switch=1
        else:
            df1= player1
            df= pd.concat([df,df1])
    return df

In [154]:
Draft_years= []
for i in range(1985,2017):
    Draft_years.append(str(i))

In [173]:
df_NCAA= NCAA_All_Together(1985)
#for year in Draft_years[1:]:
    #draft_class= NCAA_All_Together(year)
    #df_NCAA=pd.concat([df_NCAA,draft_class])

In [175]:
df_NCAA = df_NCAA.append(NCAA_All_Together(1986))
df_NCAA = df_NCAA.append(NCAA_All_Together(1987))
df_NCAA = df_NCAA.append(NCAA_All_Together(1988))
df_NCAA = df_NCAA.append(NCAA_All_Together(1989))
df_NCAA = df_NCAA.append(NCAA_All_Together(1990))
df_NCAA = df_NCAA.append(NCAA_All_Together(1991))

In [177]:
df_NCAA = df_NCAA.append(NCAA_All_Together(1992))
df_NCAA = df_NCAA.append(NCAA_All_Together(1993))
df_NCAA = df_NCAA.append(NCAA_All_Together(1994))
df_NCAA = df_NCAA.append(NCAA_All_Together(1995))

In [179]:
df_NCAA = df_NCAA.append(NCAA_All_Together(1996))
df_NCAA = df_NCAA.append(NCAA_All_Together(1997))
df_NCAA = df_NCAA.append(NCAA_All_Together(1998))
df_NCAA = df_NCAA.append(NCAA_All_Together(1999))
df_NCAA = df_NCAA.append(NCAA_All_Together(2000))

In [181]:
#df_NCAA = df_NCAA.append(NCAA_All_Together(2001))
df_NCAA = df_NCAA.append(NCAA_All_Together(2002))
#df_NCAA = df_NCAA.append(NCAA_All_Together(2003))
df_NCAA = df_NCAA.append(NCAA_All_Together(2004))
df_NCAA = df_NCAA.append(NCAA_All_Together(2005))

In [183]:
df_NCAA = df_NCAA.append(NCAA_All_Together(2006))
df_NCAA = df_NCAA.append(NCAA_All_Together(2007))
df_NCAA = df_NCAA.append(NCAA_All_Together(2008))
df_NCAA = df_NCAA.append(NCAA_All_Together(2009))
df_NCAA = df_NCAA.append(NCAA_All_Together(2010))

In [185]:
df_NCAA = df_NCAA.append(NCAA_All_Together(2011))
df_NCAA = df_NCAA.append(NCAA_All_Together(2012))
df_NCAA = df_NCAA.append(NCAA_All_Together(2013))
df_NCAA = df_NCAA.append(NCAA_All_Together(2014))
df_NCAA = df_NCAA.append(NCAA_All_Together(2015))

In [187]:
df_NCAA = df_NCAA.append(NCAA_All_Together(2016))
df_NCAA = df_NCAA.append(NCAA_All_Together(2017))
df_NCAA = df_NCAA.append(NCAA_All_Together(2018))
df_NCAA = df_NCAA.append(NCAA_All_Together(2019))

In [193]:
df_NCAA.shape

(1502, 26)

In [195]:
df_NCAA.head()

,Name,Yrs_College,School,Conf,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
0,Patrick Ewing,4,Georgetown,Big East,37,30.6,5.9,9.5,.625,,,,,,,2.8,4.3,.638,9.2,1.3,1.1,3.6,2.2,2.9,14.6,5.77
0,Wayman Tisdale,3,Oklahoma,Big 8,37,34.7,10.0,17.3,.578,,,,,,,5.2,7.4,.703,10.2,1.3,0.9,1.4,2.0,3.0,25.2,5.61
0,Benoit Benjamin,3,Creighton,MVC,32,37.3,8.1,13.8,.582,,,,,,,5.4,7.3,.738,14.1,2.1,0.8,5.1,2.8,3.1,21.5,5.36
0,Xavier McDaniel,4,Wichita State,MVC,31,36.9,11.3,20.3,.559,,,,,,,4.6,7.2,.634,14.8,2.2,1.5,1.0,3.2,3.3,27.2,4.33
0,Jon Koncak,4,SMU,SWC,33,32.8,6.6,11.2,.592,,,,,,,3.9,5.8,.667,10.7,1.3,0.4,2.8,2.5,3.5,17.2,3.32


In [196]:
df_NCAA.to_csv('NCAA_Stats_86_to_19.csv')